In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform


In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
data.index = data['PatientID']
# drop last 2 columns
data = data.drop(data.columns[-2:], axis=1)

In [3]:
data

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
PatientID,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-0.457220,0.851885,0.540813,-0.423622,0.714115,-0.564329,1.607602,0.543431,-0.870704,0.980508,...,0.673500,0.445102,0.272053,0.577840,-0.313309,0.463409,0.216033,1.875605,-0.811024,-1.250726
TCGA-A6-6142,-0.387709,0.730046,0.720710,-0.723050,0.841878,-0.463233,1.442049,0.573482,-0.817522,1.146394,...,0.287993,0.639623,0.308657,0.696497,-0.164076,0.249183,0.075877,2.179679,-0.814272,-1.311477
TCGA-5M-AAT4,-0.430112,0.766393,0.768542,-0.344121,0.560576,-0.597754,1.708495,0.419003,-0.934103,1.152953,...,0.578889,0.539531,0.399675,0.471158,-0.089670,0.701613,0.178599,1.802830,-0.844305,-1.300183
TCGA-A6-2676,-0.417006,0.964753,0.525541,-0.572356,0.777732,-0.462124,1.476457,0.640170,-0.722137,1.066126,...,0.475959,0.557490,0.094860,0.659872,-0.243587,0.122108,0.107965,2.146043,-0.946326,-1.345658
TCGA-A6-6652,-0.276354,0.766071,0.612194,-0.596442,0.814464,0.016801,1.598807,0.929635,-0.914753,1.071625,...,0.395806,0.545849,0.290805,0.229333,-0.536335,0.017757,-0.074384,2.026599,-0.776906,-0.934953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,-0.359843,0.929950,0.543955,-0.350527,0.404216,-0.566966,1.688820,0.196331,-1.050643,1.162059,...,0.802867,0.499934,0.404179,0.325667,-0.166769,0.612408,0.197617,1.686868,-0.917694,-1.251037
TCGA-RU-A8FL,-0.480253,0.931134,0.455165,-0.714430,0.685353,-0.373306,1.511218,0.747226,-0.652520,0.907837,...,0.179997,0.744851,0.062442,0.249245,-0.573766,-0.159084,-0.157464,2.173445,-0.916892,-1.098334
TCGA-G4-6303,-0.397405,0.793659,0.682602,-0.526544,0.733674,-0.456034,1.593480,0.500975,-0.982290,1.068642,...,0.514841,0.515760,0.324934,0.458768,-0.172873,0.498273,0.168238,1.823696,-0.805411,-1.238766


In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [4]:
data = data[data.index.isin(data_target.index)]

In [5]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
X = data.values
# Extract the target column for category_b_12
Y = data_target['category_stromal_34'].values
# Re-encode labels to start from zero
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)
# Instantiate the XGBoost classifier
xgb = XGBClassifier()
# Use RFE with XGBoost as the estimator
rfe = RFE(xgb, n_features_to_select=100)  # Choose top 100 features
# Fit and transform the data
X_selected = rfe.fit_transform(X, Y_encoded)


KeyboardInterrupt: 

In [16]:
# save the selected features to csv 
np.savetxt("/home/qiuaodon/Desktop/CRC_image/Best_features_XGboost_results/Top_100_features_RFE_stromal_34.csv", X_selected, delimiter=",")

In [6]:
# read in the selected features
X_selected = pd.read_csv('/home/qiuaodon/Desktop/CRC_image/Best_features_XGboost_results/Top_100_features_RFE_stromal_34.csv', header=None)

In [7]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
X = data.values
# Extract the target column for category_b_12
Y = data_target['category_stromal_34'].values
# Re-encode labels to start from zero
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)
# Instantiate and fit XGBoost
xgb = XGBClassifier()
xgb.fit(X, Y_encoded)

# Get feature importances and select the top 100 indices
importances = xgb.feature_importances_
top_100_indices = np.argsort(importances)[-100:]

# Select top 100 features
X_selected = X[:, top_100_indices]


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Extract the feature values from data
X = data.values

# Set the chunk size (number of target variables per part)
chunk_size = 40

# Loop through target columns in chunks
for chunk_start in range(0, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_features_XGboost_results/Classification_Reports_100features_XGB_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        # Loop through each target column in the current chunk
        for target_col in target_chunk:
            # Extract the target column for the current category
            Y = data_target[target_col].values

            # Encode the labels if necessary
            label_encoder = LabelEncoder()
            Y = label_encoder.fit_transform(Y)
            
            # Select top 100 features using RFE
            rfe = RFE(estimator=xgb_clf, n_features_to_select=100)  # Choose top 100 features
            X_selected = rfe.fit_transform(X, Y)
            
            # Split the data into training and testing sets
            X_train, X_test, Y_train, Y_test = train_test_split(
                X_selected, Y, test_size=0.2, random_state=42, stratify=Y
            )
            
            # Define the hyperparameter distribution for RandomizedSearchCV
            param_dist = {
                # Most important hyperparameters
                'max_depth': randint(6, 15),
                'min_child_weight': randint(1, 10),
                'subsample': uniform(0.9, 0.1),          # Higher is usually better
                'colsample_bytree': uniform(0.9, 0.1),   # Range from 0.5 to 1.0
                'learning_rate': uniform(0.01, 0.05),    # Starting from 0.01
                # Regularization hyperparameters
                'n_estimators': randint(800, 1500),
                'gamma': uniform(2, 3),
                # 'reg_alpha': uniform(0, 1),
                # 'reg_lambda': uniform(1, 5),
            }
            
            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(
                estimator=xgb_clf,
                param_distributions=param_dist,
                n_iter=60, 
                cv=5,
                scoring='f1_weighted',
                n_jobs=-1,
                verbose=1,
                random_state=42
            )
            random_search.fit(X_train, Y_train)
            
            # Get the best model from Randomized Search
            best_xgb = random_search.best_estimator_
            best_params = random_search.best_params_
            
            # Fit the model on the training data for evaluation
            best_xgb.fit(X_train, Y_train)
            
            # Evaluate on the training set
            Y_train_pred = best_xgb.predict(X_train)
            train_report_dict = classification_report(Y_train, Y_train_pred, output_dict=True)
            train_report = classification_report(Y_train, Y_train_pred)
            
            # Evaluate on the test set
            Y_test_pred = best_xgb.predict(X_test)
            test_report_dict = classification_report(Y_test, Y_test_pred, output_dict=True)
            test_report = classification_report(Y_test, Y_test_pred)
            
            # Write classification reports to the file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Test Set):\n")
            report_file.write(test_report)
            report_file.write("\n\n" + "="*80 + "\n\n")
            
            # Calculate and store test set metrics
            test_precision = precision_score(Y_test, Y_test_pred, average='weighted')
            test_recall = recall_score(Y_test, Y_test_pred, average='weighted')
            test_accuracy = accuracy_score(Y_test, Y_test_pred)
            
            # Extract test set class-specific metrics for class '1'
            test_class_1_metrics = test_report_dict.get('1', {"precision": None, "recall": None, "f1-score": None})
            
            # Calculate and store training set metrics
            train_precision = precision_score(Y_train, Y_train_pred, average='weighted')
            train_recall = recall_score(Y_train, Y_train_pred, average='weighted')
            train_accuracy = accuracy_score(Y_train, Y_train_pred)
            
            # Extract training set class-specific metrics for class '1'
            train_class_1_metrics = train_report_dict.get('1', {"precision": None, "recall": None, "f1-score": None})
            
            # Append results to chunk_results
            chunk_results.append({
                "Target Variable": target_col,
                # Test set metrics
                "Test Precision": test_precision,
                "Test Recall": test_recall,
                "Test Accuracy": test_accuracy,
                "Test Class 1 Precision": test_class_1_metrics["precision"],
                "Test Class 1 Recall": test_class_1_metrics["recall"],
                "Test Class 1 F1-Score": test_class_1_metrics["f1-score"],
                # Training set metrics
                "Train Accuracy": train_accuracy,
                "Train Class 1 F1-Score": train_class_1_metrics["f1-score"],
                # Best hyperparameters
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(
        f"/home/qiuaodon/Desktop/CRC_image/Best_features_XGboost_results/Precision_Recall_Accuracy_100features_XGB_part_{chunk_start}_{chunk_end}.xlsx",
        index=False
    )


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform


Y_category_b_12 = data_target['category_stromal_34'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    # most important hyperparameters
    'max_depth': randint(6, 15),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.9, 0.1),          # higher better
    'colsample_bytree': uniform(0.9, 0.1),   # 0.5 to 1.0 higher better
    'learning_rate': uniform(0.01, 0.05),     # 0.01
    # regularization hyperparameters
    'n_estimators': randint(1000, 1500),
    'gamma': uniform(2, 3),
    #'reg_alpha': uniform(0, 1),
    #'reg_lambda': uniform(1, 5),             # Start from 1 to avoid zero regularization

}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=150, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_stromal_34 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_stromal_34 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))
# Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.9655722635380742, 'gamma': 3.156189862896695, 'learning_rate': 0.04408068344863949, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 1416, 'subsample': 0.9496037454293407}


Fitting 5 folds for each of 150 candidates, totalling 750 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.9385097728601925, 'gamma': 4.553410014550571, 'learning_rate': 0.025846100257813885, 'max_depth': 7, 'min_child_weight': 9, 'n_estimators': 1365, 'subsample': 0.9696029796674973}
Cross-validation F1 scores on training set: [0.59825349 0.51407205 0.4841858  0.49874728 0.48648456]
Average cross-validation F1 score on training set: 0.5163486353951173
Classification Report for category_stromal_34 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       0.72      0.61      0.66       120
           1       0.61      0.66      0.63       119
           2       0.68      0.74      0.71       120

    accuracy                           0.67       359
   macro avg       0.67      0.67      0.67       359
weighted avg       0.67      0.67      0.67       359

Classification Report for category_stromal_34 with 100 featur

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform


Y_category_b_12 = data_target['category_stromal_34'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    # most important hyperparameters
    'max_depth': randint(6, 15),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.9, 0.1),          # higher better
    'colsample_bytree': uniform(0.9, 0.1),   # 0.5 to 1.0 higher better
    'learning_rate': uniform(0.01, 0.05),     # 0.01
    # regularization hyperparameters
    'n_estimators': randint(800, 1500),
    'gamma': uniform(2, 3),
    #'reg_alpha': uniform(0, 1),
    #'reg_lambda': uniform(1, 5),             # Start from 1 to avoid zero regularization

}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=60, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_stromal_34 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_stromal_34 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))
# Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.9655722635380742, 'gamma': 3.156189862896695, 'learning_rate': 0.04408068344863949, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 1416, 'subsample': 0.9496037454293407}


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.9655722635380742, 'gamma': 3.156189862896695, 'learning_rate': 0.04408068344863949, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 1416, 'subsample': 0.9496037454293407}
Cross-validation F1 scores on training set: [0.54285024 0.49941051 0.45932971 0.54278388 0.54638002]
Average cross-validation F1 score on training set: 0.5181508722678907
Classification Report for category_stromal_34 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       0.88      0.80      0.84       120
           1       0.81      0.83      0.82       119
           2       0.86      0.92      0.89       120

    accuracy                           0.85       359
   macro avg       0.85      0.85      0.85       359
weighted avg       0.85      0.85      0.85       359

Classification Report for category_stromal_34 with 100 features

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(4000, 5000),
    'learning_rate': uniform(0.01, 0.1),     # 0.01 to 0.11
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.5, 0.5),          # 0.5 to 1.0
    'colsample_bytree': uniform(0.5, 0.5),   # 0.5 to 1.0
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(1, 5),             # Start from 1 to avoid zero regularization
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.9018360384495572, 'gamma': 0.9328502944301792, 'learning_rate': 0.09925589984899778, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 4419, 'reg_alpha': 0.3562978380769749, 'reg_lambda': 5.53414220772877, 'subsample': 0.6360661246923176}
Cross-validation F1 scores on training set: [0.46427662 0.38986928 0.5047138  0.42394934 0.51685606]
Average cross-validation F1 score on training set: 0.45993302140403014
Classification Report for category_b_12 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       120
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       120

    accuracy                           1.00       359
   macro avg       1.00      1.00      1.00       359
weighted avg       1.00      1.00      1.00       359

Cl

In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5)
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.7323592099410596, 'gamma': 0.03177917514301182, 'learning_rate': 0.10329469651469865, 'max_depth': 10, 'min_child_weight': 5, 'n_estimators': 198, 'subsample': 0.8365191150830908}
Cross-validation F1 scores on training set: [0.50189167 0.41718132 0.49907191 0.39585521 0.52024264]
Average cross-validation F1 score on training set: 0.466848549262388
Classification Report for category_b_12 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       120
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       120

    accuracy                           1.00       359
   macro avg       1.00      1.00      1.00       359
weighted avg       1.00      1.00      1.00       359

Classification Report for category_b_12 with 100 features (Test Set)

In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

param_dist = {
    'n_estimators': randint(500, 1000),  # Increased upper bound
    'max_depth': randint(2, 8),          # Reduced depth
    'learning_rate': uniform(0.01, 0.09),# Lower learning rates
    'subsample': uniform(0.5, 0.5),      # Range from 0.5 to 1.0
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': randint(5, 15),  # Increased minimum
    'gamma': uniform(0, 5),              # Expanded range
    'reg_alpha': uniform(0, 1),          # Added L1 regularization
    'reg_lambda': uniform(0, 10),        # Added L2 regularization
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.5257393756249946, 'gamma': 1.3932323211830573, 'learning_rate': 0.09174392973699882, 'max_depth': 4, 'min_child_weight': 6, 'n_estimators': 552, 'reg_alpha': 0.489452760277563, 'reg_lambda': 9.856504541106007, 'subsample': 0.6210276357557503}
Cross-validation F1 scores on training set: [0.48863636 0.38843055 0.47293886 0.36705517 0.54819698]
Average cross-validation F1 score on training set: 0.45305158492273556
Classification Report for category_b_12 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       120
           1       0.98      0.97      0.98       119
           2       0.98      0.98      0.98       120

    accuracy                           0.98       359
   macro avg       0.98      0.98      0.98       359
weighted avg       0.98      0.98      0.98       359

Cla

In [11]:
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)

# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Step 1: Initialize XGBoost Classifier and RFE
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
rfe = RFE(estimator=xgb_clf, n_features_to_select=100)  # Choose top 100 features

# Fit RFE to select top 100 features
X_train_rfe = rfe.fit_transform(X_train, Y_train)
X_test_rfe = rfe.transform(X_test)


param_dist = {
    'n_estimators': randint(500, 1000),  # Increased upper bound
    'max_depth': randint(2, 8),          # Reduced depth
    'learning_rate': uniform(0.01, 0.09),# Lower learning rates
    'subsample': uniform(0.5, 0.5),      # Range from 0.5 to 1.0
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': randint(5, 15),  # Increased minimum
    'gamma': uniform(0, 5),              # Expanded range
    'reg_alpha': uniform(0, 1),          # Added L1 regularization
    'reg_lambda': uniform(0, 10),        # Added L2 regularization
}

# Perform Randomized Search with cross-validation on the selected features
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train_rfe, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train_rfe, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Step 3: Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train_rfe, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train_rfe)
print("Classification Report for category_b_12 with 100 selected features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test_rfe)
print("Classification Report for category_b_12 with 100 selected features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.5866471600354228, 'gamma': 0.7821852133554302, 'learning_rate': 0.03252186083481358, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 547, 'reg_alpha': 0.18286599710730733, 'reg_lambda': 9.346139973397097, 'subsample': 0.8191352969216752}
Cross-validation F1 scores on training set: [0.56608155 0.44555012 0.51604938 0.51551307 0.51535495]
Average cross-validation F1 score on training set: 0.5117098133827519
Classification Report for category_b_12 with 100 selected features (Train Set):
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       120
           1       0.98      0.97      0.98       119
           2       0.98      0.98      0.98       120

    accuracy                           0.98       359
   macro avg       0.98      0.98      0.98       359
weighted avg       0.98      0.98      0.98    

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint, uniform

# Assuming 'data' and 'data_target' are your feature matrix and target DataFrame respectively
# X contains feature values
X = data.values

# Set the chunk size (number of target variables per part)
chunk_size = 40

# Loop through target columns in chunks
for chunk_start in range(0, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/XGboost_results/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        # Iterate over each target column in the chunk
        for target_col in target_chunk:
            # Extract the target column for the current category
            Y = data_target[target_col].values

            # Encode labels if they are not numerical
            label_encoder = LabelEncoder()
            Y_encoded = label_encoder.fit_transform(Y)

            # Initialize the XGBoost Classifier
            xgb_clf = xgb.XGBClassifier(
                objective='multi:softmax', 
                random_state=42, 
                use_label_encoder=False,
                eval_metric='mlogloss'
            )

            # Select top 100 features using RFE
            rfe = RFE(estimator=xgb_clf, n_features_to_select=100)
            X_selected = rfe.fit_transform(X, Y_encoded)
            
            # Split the data into training and testing sets with stratification
            X_train, X_test, Y_train, Y_test = train_test_split(
                X_selected, Y_encoded, test_size=0.2, random_state=42, stratify=Y_encoded
            )

            # Define the hyperparameter distribution for XGBoost
            param_dist = {
                'n_estimators': randint(500, 1000),  # Increased upper bound
                'max_depth': randint(2, 8),          # Reduced depth
                'learning_rate': uniform(0.01, 0.09),# Lower learning rates
                'subsample': uniform(0.5, 0.5),      # Range from 0.5 to 1.0
                'colsample_bytree': uniform(0.5, 0.5),
                'min_child_weight': randint(5, 15),  # Increased minimum
                'gamma': uniform(0, 5),              # Expanded range
                'reg_alpha': uniform(0, 1),          # Added L1 regularization
                'reg_lambda': uniform(0, 10),        # Added L2 regularization
            }

            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(
                estimator=xgb_clf,
                param_distributions=param_dist,
                n_iter=50, 
                cv=5, 
                scoring='f1_weighted', 
                n_jobs=-1, 
                verbose=1, 
                random_state=42
            )
            random_search.fit(X_train, Y_train)

            # Get the best model from Randomized Search
            best_xgb = random_search.best_estimator_
            best_params = random_search.best_params_

            # Fit the best model on the training data
            best_xgb.fit(X_train, Y_train)

            # Evaluate on the training set
            Y_train_pred = best_xgb.predict(X_train)
            train_report = classification_report(Y_train, Y_train_pred)

            # Evaluate on the test set
            Y_test_pred = best_xgb.predict(X_test)
            test_report_dict = classification_report(Y_test, Y_test_pred, output_dict=True)
            test_report = classification_report(Y_test, Y_test_pred)

            # Write classification reports to the file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Test Set):\n")
            report_file.write(test_report)
            report_file.write("\n\n" + "="*80 + "\n\n")

            # Calculate and store precision, recall, and accuracy
            precision = precision_score(Y_test, Y_test_pred, average='weighted')
            recall = recall_score(Y_test, Y_test_pred, average='weighted')
            accuracy = accuracy_score(Y_test, Y_test_pred)

            # Extract precision, recall, and f1-score specifically for class '1' (if it exists)
            class_labels = label_encoder.classes_
            class_indices = label_encoder.transform(class_labels)
            class_1_index = np.where(class_indices == 1)[0]

            if class_1_index.size > 0:
                class_1_label = str(class_indices[class_1_index[0]])
                class_1_metrics = test_report_dict.get(class_1_label, {"precision": None, "recall": None, "f1-score": None})
            else:
                class_1_metrics = {"precision": None, "recall": None, "f1-score": None}

            chunk_results.append({
                "Target Variable": target_col,
                "Precision": precision,
                "Recall": recall,
                "Accuracy": accuracy,
                "Class 1 Precision": class_1_metrics["precision"],
                "Class 1 Recall": class_1_metrics["recall"],
                "Class 1 F1-Score": class_1_metrics["f1-score"],
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(
        f"/home/qiuaodon/Desktop/CRC_image/XGboost_results/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx",
        index=False
    )
